In [1]:
!pip install wget
import os
import torch
import pandas as pd
import numpy as np
import torch
from torch import nn


ssh_config = """
Host github.com
  IdentityFile ~/.ssh/github.pem
  User davipeag
  StrictHostKeyChecking no
"""

if os.name == 'nt':
  base_path = ""
  REPO_DIR = "."
  STORE_DIR ="." 
  print("Windows")
else:
  print("Unix-like")
  REPO_DIR = "/tmp/HeartRateRegression"
  from google.colab import drive
  drive.mount('/content/drive')
  GIT_PATH = "/content/drive/My\ Drive/deeplearning_project/github.pem"
  DATA_DIR = os.path.join(REPO_DIR, "repo")
  STORE_DIR ="/content/drive/My Drive/deeplearning_project/" 
  !mkdir ~/.ssh
  !cp -u {GIT_PATH} ~/.ssh/
  !chmod u=rw,g=,o= ~/.ssh/github.pem
  !echo "{ssh_config}" > ~/.ssh/config
  !chmod u=rw,g=,o= ~/.ssh/config
  ! (cd /tmp && git clone git@github.com:davipeag/HeartRateRegression.git)
  ! (cd {REPO_DIR} && git pull )
  import sys
  sys.path.append(REPO_DIR)

def git_pull():
  ! (cd {REPO_DIR} && git pull )

git_pull()

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=ef4480e26ee6625710062994831d0e1c22091c944c3fccc82b0454803aa2190b
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Unix-like
Mounted at /content/drive
Cloning into 'HeartRateRegression'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 802 (delta 46), reused 36 (delta 15), pack-reused 719
Receiving objects: 100% (802/802), 88.08 MiB | 30.32 MiB/s, done.
Resolving deltas: 100% (502/502), done.
Already up to date.
Already up to date.


In [2]:
import random
import torch
from torch import nn
import numpy as np
args = {
    'epoch_num': 250,     # Number of epochs.
    'lr': 1.0e-3,           # Learning rate.
    'weight_decay': 10e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 0,     # Number of workers on data loader.
    'batch_size': 128,     # Mini-batch size. 128
    'batch_test': 248,     # size of test batch
    'window': 15,
    'initial_window':5,
    'clip_norm': 6.0,     # Upper limit on gradient L2 norm ###
}
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

SEED = 1234
def reset_seeds():
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

reset_seeds()

cuda


In [3]:
from data_utils import (FormatIeee,  IeeeExtractor)

SUBJECTS = list(range(1,13))

extractor = IeeeExtractor(DATA_DIR)
formatter = FormatIeee()
dfs_train = [formatter.transform(extractor.extract_subject(i)) for i in SUBJECTS]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  import sys


In [4]:
git_pull()

import importlib

import PPG

from PPG import FullTrainer

importlib.reload(PPG.AttentionDefaults)
importlib.reload(PPG)
importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(PPG.Models)
importlib.reload(PPG.NoHrPceLstmModel)
importlib.reload(PPG.TrainerXY)
importlib.reload(PPG.TrainerIS)
importlib.reload(PPG.FullTrainer)


Already up to date.


<module 'PPG.FullTrainer' from '/tmp/HeartRateRegression/PPG/FullTrainer.py'>

In [ ]:
import matplotlib.pyplot as plt
import pickle
# fchoice = {'val_sub': 4,
#   'ts_sub': 0,
#   'batch_size': 64,
#   'weight_decay': 0,
#   'lr': 0.001,
#   'nattrs': 5,
#   'bvp_count': 16,
#   'dropout_rate': 0.25,
#   'lstm_input': 128,
#   'lstm_size': 64,
#   'ts_h_size': 64
#   }
def compute_ensemble(results):
  ps = [v["predictions"][1].reshape(-1).numpy() for v in results]
  ys = [v["predictions"][0].reshape(-1).numpy() for v in results]

  for i in range(1, len(ys)-1):
    assert np.all(ys[i] == ys[i-1])

  s = ps[0]
  for p in ps[1:]:
    s = s + p

  a = s/len(ps)
  y = ys[0]

  plt.plot(a)
  plt.plot(y)

  return np.mean(np.abs(a - y))


fchoice = {'val_sub': 4,
  'ts_sub': 0,
  'batch_size': 64,
  'weight_decay': 0,
  'lr': 0.0001,
  'lin_dropout': 0,
  'lin_size': 16,
  'nlin_layers': 2,
  'feedforward_expansion': 1,
  'nhead': 4,
  'ndec_layers': 2,
  'nenc_layers': 2,
  'conv_dropout': 0,
  'nconv_layers': 2,
  'conv_filters': 128,
  'nfeatures': 5
}


from PPG import UtilitiesDataXY


aresults = list()
for ts_sub in [0,1,2,3, 4,5,6,7,8,9,10,11]:
  dresults = list()
  for i in range(7):
    filename = f"ieee_train_ts_attention_{ts_sub}_{i}.pkl"
    save_path = os.path.join(STORE_DIR, filename)
    try:
      with open(save_path , "rb") as f:
        out = pickle.load(f)
    except FileNotFoundError:
      full_trainer = FullTrainer.IeeeJointValAttentionFullTrainer(dfs_train, args["device"])
    else:
      dresults.append(out)
      continue
    # try:
    fchoice["ts_sub"] = ts_sub
    out = full_trainer.train(**fchoice)
    print(out["args"], out["metric"])
    dresults.append(out)
    with open(save_path, "wb") as f:
      pickle.dump(out, f)
    # except RuntimeError as e:
    #   if isinstance(e, KeyboardInterrupt):
    #     raise e
    #   else:
    #     print("####")
    #     print(f"Failed: {choice}")
    #     print("###")
  print(f"subject: {ts_sub}")
  print(f"TS:{compute_ensemble(dresults)}")
  aresults.append(dresults)


subject: 0
TS:3.5837992436594845
subject: 1
TS:3.5164453237299105
subject: 2
TS:2.901921563960521
subject: 3
TS:4.78613605398583
subject: 4
TS:1.8175862372289742
subject: 5
TS:2.617658858014949
best val epoch: 1
[1/30]: loss_train: 4.259 loss_val 4.179 loss_ts 3.669
best val epoch: 2
[2/30]: loss_train: 3.473 loss_val 3.409 loss_ts 2.747
best val epoch: 4
[4/30]: loss_train: 3.436 loss_val 3.394 loss_ts 2.421
best val epoch: 6
[6/30]: loss_train: 3.454 loss_val 3.393 loss_ts 2.462
best val epoch: 7
[7/30]: loss_train: 3.390 loss_val 3.338 loss_ts 2.336
best val epoch: 8
[8/30]: loss_train: 3.376 loss_val 3.327 loss_ts 2.305
best val epoch: 11
[11/30]: loss_train: 3.350 loss_val 3.293 loss_ts 2.305
best val epoch: 12
[12/30]: loss_train: 3.312 loss_val 3.260 loss_ts 2.241
best val epoch: 13
[13/30]: loss_train: 3.282 loss_val 3.232 loss_ts 2.204
best val epoch: 15
[15/30]: loss_train: 3.239 loss_val 3.179 loss_ts 2.389
best val epoch: 17
[17/30]: loss_train: 3.109 loss_val 3.050 loss_ts